# Upload data from Excel

### Import enviPath packages, getpass for passwords, and pandas for data cleaning

In [102]:
from enviPath_python.enviPath import *
from enviPath_python.objects import *
import getpass
import pandas as pd

### Access password-protected package

In [103]:
if __name__ == "__main__":
    eP = enviPath('https://envipath.org')
    username = "slr257"
    eP.login(username, getpass.getpass(prompt="Password: ", stream=None))

    pkg = Package(eP.requester, id="https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e") # Currently the upload tests package

Password:  ········


### Read in excel data from template
each sheet is a separate dataframe

In [104]:
folder_name = "PFAS_excel_uploadfiles/"
file_name = "PFAS_upload_template_v5.xlsx"

In [129]:
# read in excel data from template using pandas
cmpds = pd.read_excel(folder_name + file_name, sheet_name = "Compounds")
conn = pd.read_excel(folder_name + file_name, sheet_name = "Connectivity")
scn = pd.read_excel(folder_name + file_name, sheet_name = "Scenario")
kin_conf = pd.read_excel(folder_name + file_name, sheet_name = "Kinetics_Confidence")

# Scenario_Name = "Rhoads, K.R. et al, 2008"
Scenario_Name = "test_AIobjects"
DOI = "10.1021/es702866c"

# running this generates a warning because there are drop-down lists in excel from the "data validation" extension that do not carry over into the dataframes here

C:\Users\strich\AppData\Local\anaconda3\envs\enviPath_test_12324\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\strich\AppData\Local\anaconda3\envs\enviPath_test_12324\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\strich\AppData\Local\anaconda3\envs\enviPath_test_12324\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### Add Compounds

First work with the compounds data frame to upload nodes

In [130]:
# define parent node
parent = cmpds.loc[cmpds['Type'] == 'Parent']

# creat root node
# pw = Pathway.create(pkg, smiles = parent.SMILES[0], name = parent.Name[0], root_node_only = True)
pw = Pathway.create(pkg, smiles = parent.SMILES[0], name = "test_pathway_240703", root_node_only = True)

# library with SMILES and URLs for each node
map_smiles_to_URL = {"SMILES": [parent.SMILES[0]], "URL": [pw.get_nodes()[0].get_id()]} 

# related nodes
TPs = cmpds.loc[cmpds['Type'] != 'Parent']

for index in TPs.index:
    #if Type!="Parent":
        #Create Node
    row = TPs.loc[index]
    n = Node.create(pw, smiles = row.SMILES, name = row.Name)
    pw.add_node(smiles = row.SMILES, name = row.Name)
    # print(n.get_smiles() + " is not the same as " + row.SMILES )
    map_smiles_to_URL["SMILES"].append(row.SMILES)
    map_smiles_to_URL["URL"].append(n.get_id())
    print(f"Accessing compound {row.Name} with SMILES {row.SMILES}")
    
temporary_df = pd.DataFrame.from_dict(map_smiles_to_URL, orient="columns")
cmpds = cmpds.merge(temporary_df, left_on="SMILES", right_on="SMILES")

Accessing compound 6:2 FTUSA with SMILES FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)/C=C/S(O)(=O)=O)F
Accessing compound Ketone-6:2 FTSA with SMILES FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(CS(O)(=O)=O)=O)F
Accessing compound OH-6:2 FTSA with SMILES FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(O)CS(O)(=O)=O)F
Accessing compound 5:3 FTU amide with SMILES FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)/C=C/C(N)=O)F
Accessing compound 5:3 FT amide-AcA with SMILES FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCC(NCC(O)=O)=O)F
Accessing compound 6:2 FTUCA with SMILES C(=C(\C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)/F)/C(=O)O
Accessing compound 5:2 FTUCA with SMILES FC(F)(C(F)(F)C(F)(F)C(F)(F)/C(F)=C/C(O)=O)F
Accessing compound 4:2 FTUCA with SMILES FC(F)(C(F)(F)C(F)(F)/C(F)=C/C(O)=O)F
Accessing compound 6:2 FTOH with SMILES C(CO)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F
Accessing compound PFHpA with SMILES C(=O)(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)O
Accessing compound 5:2 FT Ketone with SMILES CC(=O)C(C(C(C(C(F

### merge kinetics dataframe with compounds dataframe using the URLs
### create connectivity dataframe

In [131]:
kin_conf = kin_conf.merge(temporary_df, left_on = "SMILES", right_on = "SMILES")
conn = pd.read_excel(folder_name + file_name, sheet_name = "Connectivity")

C:\Users\strich\AppData\Local\anaconda3\envs\enviPath_test_12324\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### add edges based on connectivity dataframe

In [132]:
# now to add connectivity using edge info
for index in conn.index:
    #if Type!="Parent":
        #Create Node
    row = conn.loc[index]
    #pw.add_node(smiles = row.SMILES, name = row.Name)
    #print(f"Reactant smiles {row.Reactant_SMILES} with product smiles {row.Product_SMILES}")
    edge = row.Reactant_SMILES + '>>' + row.Product_SMILES

    # boolean value to add multistep attribute
    if pd.isnull(row.Multistep) == False:
        multi = row.Multistep
    else:
        multi = False
    print(edge)
    pw.add_edge(smirks = edge, multistep = multi)
# access connectivity dataframe

# concatenate >> between reactants and products

C(CS(=O)(=O)O)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F>>FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)/C=C/S(O)(=O)=O)F
C(CS(=O)(=O)O)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F>>FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(O)CS(O)(=O)=O)F
C(CS(=O)(=O)O)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F>>C(CO)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F
FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)/C=C/S(O)(=O)=O)F>>FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(CS(O)(=O)=O)=O)F
FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(O)CS(O)(=O)=O)F>>FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(CS(O)(=O)=O)=O)F
FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(CS(O)(=O)=O)=O)F>>C(=O)(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)O
C(CO)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F>>C(C(=O)O)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F
C(C(=O)O)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F>>C(=C(\C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)/F)/C(=O)O
C(=C(\C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)/F)/C(=O)O>>CC(=O)C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F

### Create Main Scenario Object

In [133]:
# main scenario - aka "base"
#base = scn[(scn['Environment Type'] == 'Activated Sludge')]
base = scn

### Checking Scenario Information and matching to enviPath objects

In [134]:
# this only works with the other fils is named utils...
import utils

# need to reload package when it's updated
from importlib import reload
reload(utils)

<module 'utils' from 'C:\\Users\\strich\\OneDrive - Universität Zürich UZH\\Documents\\data curation - EAWAG-PFAS package\\utils.py'>

### subset dictionary of functions to choose which functions to run to add AI objects

In [135]:
subset_dict = {k: utils.dict_ai[k] for k in base["Parameter"]}
print(subset_dict)

{'Addition of nutrients': <class 'enviPath_python.objects.NutrientsAdditionalInformation'>, 'Ammonia Uptake Rate (AUR)': <class 'enviPath_python.objects.AmmoniaUptakeRateAdditionalInformation'>, 'Biological treatment technology': <class 'enviPath_python.objects.BiologicalTreatmentTechnologyAdditionalInformation'>, 'Bioreactor': <class 'enviPath_python.objects.BioreactorAdditionalInformation'>, 'Dissolved organic carbon (DOC)': <class 'enviPath_python.objects.DissolvedOrganicCarbonAdditionalInformation'>, 'Dissolved oxygen concentration': <class 'enviPath_python.objects.DissolvedOxygenConcentrationAdditionalInformation'>, 'Final compound concentration': <class 'enviPath_python.objects.FinalCompoundConcentrationAdditionalInformation'>, 'Initial amount of sludge in bioreactor': <class 'enviPath_python.objects.OriginalSludgeAmountAdditionalInformation'>, 'Inoculum source': <class 'enviPath_python.objects.InoculumSourceAdditionalInformation'>, 'Name of Sampling Location': <class 'enviPath_p

In [136]:
# This function is really nice!!
# this should only run the add function if the variable k is in the function dictionary
additional_info_list = []
for k in subset_dict.keys():
    # print(k)
    # if the key (k) is in the dictionary, run the function to add the AI object
    if k in utils.dict_fun:
        utils.dict_fun[k](base, additional_info_list)
        # print(additional_info_list)
    else:
        pass

### Create a scenario object in enviPath

In [137]:
main_scenario = Scenario.create(pkg, name = Scenario_Name, description = "Reference:" + DOI ,additional_information=additional_info_list)

In [138]:
# add scenario to pathway
pw.add_scenario(main_scenario)

In [139]:
pw.get_nodes()

[Node: PFBA (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/b625346d-e737-4cc7-9e69-67149299814b/node/f51d5879-7b70-4431-afce-898564bc3ea1),
 Node: 5:3 FT amide-AcA (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/b625346d-e737-4cc7-9e69-67149299814b/node/9efb7e2d-6a7a-4635-9bc4-b8b3483d7ccd),
 Node: 6:2 FTUCA (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/b625346d-e737-4cc7-9e69-67149299814b/node/08a5b3ba-c62c-4fb6-b33a-b2056db62b53),
 Node: 5:2 FTUCA (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/b625346d-e737-4cc7-9e69-67149299814b/node/f52881cc-95e6-4636-ad8b-aafeca2de373),
 Node: structure 0000006 (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/b625346d-e737-4cc7-9e69-67149299814b/node/14a4741c-ad40-4e2a-a2b3-5eae856517e8),
 Node: PFPeA (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/b625346d-e737-4cc7-9e69-67149299814b/

### Create related scenarios using kinetics and confidence dataframes

In [146]:
# This set of conditions compares the names of the compounds in the table with the names + aliases of compounds in the nodes and adds related scenarios accordingly
node_list = kin_conf
for index in node_list.index:
    # print(index)
    node_info = node_list.loc[index]
    node = Node(eP.requester, id=node_info.URL)
            
    # print(node_info)
    
    ai_list_update = []

    if (not pd.isna(node_info["Confidence Level_Schy"])):
        ai1 = ConfidenceLevelAdditionalInformation()
        ai1.set_radioconfidence(node_info["Confidence Level_Schy"])
        ai_list_update.append(ai1)
        
    if (not pd.isna(node_info[["Model", "Value"]]).any()):
        if(node_info["Parameter"] == "Half-life"):
            ai2 = HalfLifeAdditionalInformation()
            ai2.set_lower(node_info["Value"])
            ai2.set_upper(node_info["Value"])
            ai2.set_model(node_info["Model"])
            ai_list_update.append(ai2)
        
    if (not pd.isna(node_info[["Model", "Corrected", "Comment"]]).any()):
        if(node_info["Parameter"] == "Rate constant"):
            ai3 = RateConstantAdditionalInformation()
            ai3.set_rateconstantlower(node_info["Value"])
            ai3.set_rateconstantupper(node_info["Value"])
            ai3.set_rateconstantorder(node_info["Model"])
            ai3.set_rateconstantcorrected(node_info["Corrected"])
            ai3.set_rateconstantcomment(node_info["Comment"])
            ai_list_update.append(ai3)


    if (not len(ai_list_update) == 0):
        referral_scenario = Scenario.create(pkg, referring_scenario_id=main_scenario.get_id(), additional_information=[])
        referral_scenario.update_scenario(additional_information=ai_list_update)
        node.add_scenario(referral_scenario)
                

### deleting pathways

In [31]:
pws_to_delete = ["https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/4a86eddb-e3ed-4b1b-ae88-80b95a70af26",
                "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/92a66fe7-dadd-4b83-82f4-6f1f1a687d3a",
                "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/97ecec79-2803-4e8b-a9dc-47685394c85f"]


In [64]:
pws_to_delete =["https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/1e683118-362a-40e7-ad3e-f36b86d47803"]

In [65]:
for pw_URL in pws_to_delete:
    pw = Pathway(eP.requester, id=pw_URL)
    pw.delete()

# pw.delete()

### deleting scenarios

In [41]:
scns_to_delete = ["https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/4d575c5c-d1d3-463f-a303-c08f7ae13d6f",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/783e33ed-0a7b-4fc4-bf48-7a37fe62c4c3",
                "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/2149fd26-a24c-4c14-ba6a-f0085a2a60f6",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/1d169ee3-1aa1-4435-82cb-86ea8939aa9c",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/afbe93d8-23e8-426f-a352-fb60a1a2a604",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/ed4859e8-b65e-4c6f-93b1-863a415722bd",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/8f098770-2d77-4d27-accc-3773ac2732d2",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/2ff0a0c0-cd56-4539-8f86-410674beb20a",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/3381330e-a56a-40be-a2af-966641ba0013"]

In [20]:
scns_to_delete = ["https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/cfa0edef-2e16-4b88-a596-5364c284af99"]

In [42]:
for scn_URL in scns_to_delete:
    scn = Scenario(eP.requester, id=scn_URL)
    scn.delete()